In [200]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from datetime import datetime
from bs4 import BeautifulSoup
import pandas as pd
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options
import sys

In [201]:
def search(driver, date):
    """
    Date in mm/dd/yyyy
    """
    try:
        driver.get("https://merolagani.com/Floorsheet.aspx")
        try:
            alert = driver.switch_to.alert
            print(f'Alert detected {alert.text}')
            alert.dismis()
        except Exception as e:
            pass
        date_element_path = '/html/body/form/div[4]/div[4]/div/div/div[1]/div[4]/input'
        search_element_path2 = '/html/body/form/div[4]/div[4]/div/div/div[2]/a[1]'
        date_input = driver.find_element(By.XPATH, date_element_path)
        search_btn = driver.find_element(By.XPATH, search_element_path2)
        # date_input = driver.find_element_by_xpath()
        # search_btn = driver.find_element_by_xpath()
        date_input.send_keys('02/13/2025')
        print(search_btn)
        search_btn.click()
    except Exception as e:
        print(e)
    # if driver.find_elements(By.XPATH, "//*[contains(text(), 'Could not find floorsheet matching the search criteria')]"):
    #     print("No data found for the given search.")
    #     print("Aborting script ......")
    #     sys.exit()

In [203]:
def get_page_table(driver, table_class):
    soup = BeautifulSoup(driver.page_source,'html')
    table = soup.find("table", {"class":table_class})
    tab_data = [[cell.text.replace('\r', '').replace('\n', '') for cell in row.find_all(["th","td"])]
                        for row in table.find_all("tr")]
    df = pd.DataFrame(tab_data)
    return df

In [204]:
def scrape_data(driver, date):
    start_time = datetime.now()
    search(driver,'02/13/2025')
    df = pd.DataFrame()
    while True:
        page_table_df = get_page_table(driver, table_class="table table-bordered table-striped table-hover sortable")
        print(type(page_table_df))
        
        if page_table_df is not None and not page_table_df.empty:
            df = pd.concat([df, page_table_df], ignore_index=True)
        # df = df.append(page_table_df, ignore_index = True)
        print(df.head())
        try:
            next_btn = driver.find_element(By.LINK_TEXT, 'Next')
            # next_btn = driver.find_element_by_link_text('Next')
            driver.execute_script("arguments[0].click();", next_btn)
        except NoSuchElementException:
            break
    print(f"Time taken to scrape: {datetime.now() - start_time}")    
    return df

In [205]:
def clean_df(df):
    new_df = df.drop_duplicates(keep='first') # Dropping Duplicates
    new_header = new_df.iloc[0] # grabing the first row for the header
    new_df = new_df[1:] # taking the data lower than the header row
    new_df.columns = new_header # setting the header row as the df header
    new_df.drop(["#"], axis=1, inplace=True)
    new_df["Rate"] = new_df["Rate"].apply(lambda x:float(x.replace(",", ""))) # Convert Rate to Float
    new_df["Amount"] = new_df["Amount"].apply(lambda x:float(x.replace(",", ""))) # Convert Amount to Float
    return new_df

In [206]:
options = Options()
options.headless = True
options.add_argument('--disable-notifications')
driver = webdriver.Chrome(options=options) # Start Browser

date = datetime.today().strftime('%m/%d/%Y') # Get today's date
search(driver, '02/13/2025') # Search the webpage
df = scrape_data(driver, date) # Scraping
final_df = clean_df(df) # Cleaning

<selenium.webdriver.remote.webelement.WebElement (session="67c346b116fb205839b6eb18b20b4c70", element="f.A20EF2365D53FA332EFF7702A684A706.d.1FF8106380D15BB9A749D0A165CC8821.e.96")>
Alert Text: Do you want to receive notifications?
Message: unexpected alert open: {Alert text : Do you want to receive notifications?}
  (Session info: chrome=133.0.6943.98)
Stacktrace:
#0 0x6259eca4fbba <unknown>
#1 0x6259ec4ed790 <unknown>
#2 0x6259ec58b7a0 <unknown>
#3 0x6259ec564823 <unknown>
#4 0x6259ec530a88 <unknown>
#5 0x6259ec531bf1 <unknown>
#6 0x6259eca1915b <unknown>
#7 0x6259eca1d0e2 <unknown>
#8 0x6259eca0601c <unknown>
#9 0x6259eca1dcd4 <unknown>
#10 0x6259ec9ea48f <unknown>
#11 0x6259eca3e4f8 <unknown>
#12 0x6259eca3e6c9 <unknown>
#13 0x6259eca4ea36 <unknown>
#14 0x7d83dc49caa4 <unknown>
#15 0x7d83dc529c3c <unknown>

<class 'pandas.core.frame.DataFrame'>
   0                 1       2      3       4         5         6           7
0  #     Transact. No.  Symbol  Buyer  Seller  Quantity      R

In [207]:
final_df.head()

,Transact. No.,Symbol,Buyer,Seller,Quantity,Rate,Amount
1,2025021305016233,BBC,96,14,10,5587.0,55870.0
2,2025021305015830,BBC,22,1,30,5520.0,165600.0
3,2025021305015829,BBC,6,1,10,5522.0,55220.0
4,2025021305015828,BBC,26,1,10,5523.0,55230.0
5,2025021305015827,BBC,57,1,10,5523.0,55230.0


In [208]:
file_name = date.replace("/", "_")
final_df.to_csv(f"data/{file_name}.csv", index=False) # Save file